In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import (
    layers,
    models,
    datasets,
    callbacks,
    losses,
    optimizers,
    metrics,
)
import keras
from tensorflow.keras.callbacks import TensorBoard
from tensorboard.plugins.hparams import api as hp
from tensorflow import keras
from keras.layers import Input, Dense, Lambda
from itertools import product

from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [3]:
train_data = pd.read_csv("../data/interim/cut_data.csv")
val_data = pd.read_csv("../data/interim/split/sorted_val_data_mm.csv")

In [4]:
train_data_array = train_data.values
val_data_array = val_data.values

In [5]:
# https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams

list_of_layers = [64, 128, 256, 512]
list_of_latent = [2, 4, 16, 32, 64]
list_of_boolean = [True, False]
list_of_kl_loss_rate =  [0.0001, 0.1, 1.0, 10.0]
list_of_binary_weight =  [0.1, 1.0, 10.0]


hyperparameters_product = product(
    list_of_layers, list_of_layers, list_of_latent,
    list_of_kl_loss_rate, list_of_binary_weight, list_of_boolean, list_of_boolean, 
)
# list_of_drop_out,
# Calculate the total number of combinations
number_of_combinations = len(list(hyperparameters_product))

print("Total number of combinations:", number_of_combinations)

Total number of combinations: 3840


In [6]:
HP_NUM_embedded = hp.HParam("num_embedded", hp.Discrete(list_of_latent))
HP_NUM_UNITS = hp.HParam("num_units", hp.Discrete(list_of_layers))
HP_NUM_UNITS1 = hp.HParam("num_units1", hp.Discrete(list_of_layers))
#HP_BOOL_BATCH = hp.HParam("bool_batch", hp.Discrete(list_of_boolean))
HP_BOOL_2_LAYER = hp.HParam("bool_layer_2", hp.Discrete(list_of_boolean))
HP_BOOL_4_LAYER = hp.HParam("bool_layer_4", hp.Discrete(list_of_boolean))
# HP_DROP_OUT_BOOL = hp.HParam("drop_out", hp.Discrete(list_of_boolean))
# HP_DROP_OUT_RATE = hp.HParam("drop_out_rate", hp.Discrete(list_of_drop_out))


HP_KL_LOSS_RATE = hp.HParam("KL_LOSS_RATE", hp.Discrete(list_of_kl_loss_rate))
HP_BINARY_WEIGHT = hp.HParam("BINARY_WEIGHT", hp.Discrete(list_of_binary_weight))

VAL_LOSS = "val_loss"
VAL_MEAN_BINARY_VALUE = "val_mean_binary_value"
VAL_MEAN_CONT_DIFF = "val_mean_abs_diff_cont"
VAL_MEAN_CLASS_DIFF = "val_mean_abs_diff_class"
VAL_MEAN_KL_LOSS = "val_mean_kl_loss"
VAL_MEAN_KL_DIV = "val_mean_kl_div"
VAL_LOSS_SAMPLE = "val_loss_sample"
VAL_MEAN_BINARY_VALUE_SAMPLE = "val_mean_binary_value_sample"
VAL_MEAN_CONT_DIFF_SAMPLE = "val_mean_abs_diff_cont_sample"
VAL_MEAN_CLASS_DIFF_SAMPLE = "val_mean_abs_diff_class_sample"
VAL_MEAN_KL_DIV_SAMPLE = "val_mean_kl_div_sample"


# the other one was stored at logs/hparam_tuning_vae_from_base1
path = "logs/hparam_tuning_vae_extended_corrected/"


with tf.summary.create_file_writer(path).as_default():
    hp.hparams_config(
        hparams=[
            HP_NUM_embedded,
            # HP_BOOL_BATCH,
            HP_BOOL_2_LAYER,
            HP_BOOL_4_LAYER,
            HP_NUM_UNITS,
            HP_NUM_UNITS1,
            # HP_DROP_OUT_BOOL,
            # HP_DROP_OUT_RATE,
            HP_KL_LOSS_RATE,
            HP_BINARY_WEIGHT,
        ],

        metrics=[
            hp.Metric(VAL_LOSS, display_name="val_loss"),
            hp.Metric(VAL_MEAN_BINARY_VALUE, display_name="val_mean_binary_value",),
            hp.Metric(VAL_MEAN_CONT_DIFF, display_name="val_mean_abs_diff_cont"),
            hp.Metric(VAL_MEAN_CLASS_DIFF, display_name="val_mean_abs_diff_class"),
            hp.Metric(VAL_MEAN_KL_LOSS, display_name= "val_mean_kl_loss"),
            hp.Metric(VAL_MEAN_KL_DIV, display_name = "val_mean_kl_div"),
            hp.Metric(VAL_LOSS_SAMPLE, display_name="val_loss_sample"),
            hp.Metric(VAL_MEAN_BINARY_VALUE_SAMPLE, display_name="val_mean_binary_value_sample"),
            hp.Metric(VAL_MEAN_CONT_DIFF_SAMPLE, display_name="val_mean_abs_diff_cont_sample"),
            hp.Metric(VAL_MEAN_CLASS_DIFF_SAMPLE, display_name="val_mean_abs_diff_class_sample"),
            hp.Metric(VAL_MEAN_KL_DIV_SAMPLE, display_name="val_mean_kl_div_sample"),  
        ],
    )

In [7]:
def build_model(hparams):
    class Sampling(keras.layers.Layer):
        def call(self, inputs):
            z_mean, z_log_var = inputs
            batch = tf.shape(z_mean)[0]
            dim = tf.shape(z_mean)[1]
            # epsilon = gumbel_sample((batch, dim))
            epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
            return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    latent_dim = hparams[HP_NUM_embedded]
    encoder_inputs = Input(shape=(31), name="input_layer")

    x = Dense(hparams[HP_NUM_UNITS], activation="relu", name="h1")(encoder_inputs)

    #if hparams[HP_BOOL_BATCH] == True:
    x = layers.BatchNormalization()(x)
    # if hparams[HP_DROP_OUT_BOOL] == True:
    #     x = layers.Dropout(0.3)(x)

    if hparams[HP_BOOL_2_LAYER] == True:
        x = Dense(hparams[HP_NUM_UNITS1], activation="relu", name="h2")(x)
        # Split x3 into two halves
        half_size = hparams[HP_NUM_UNITS1] // 2
    else:
        half_size = hparams[HP_NUM_UNITS] // 2

    x1_first_half = Lambda(lambda x: x[:, :half_size], name="select_z_mean")(x)
    x1_second_half = Lambda(lambda x: x[:, half_size:], name="select_z_var")(x)

    z_mean = Dense(latent_dim, name="z_mean")(x1_first_half)
    z_log_var = Dense(latent_dim, name="z_log_var")(x1_second_half)
    z = Sampling(name="Sampling")([z_mean, z_log_var])

    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

    # Decoder
    latent_inputs = keras.Input(shape=(latent_dim,))
    x = Dense(hparams[HP_NUM_UNITS1], activation="relu", name="h4")(latent_inputs)

    #if hparams[HP_BOOL_BATCH] == True:
    x = layers.BatchNormalization()(x)
    # if hparams[HP_DROP_OUT_BOOL] == True:
    #     x = layers.Dropout(0.3)(x)

    if hparams[HP_BOOL_4_LAYER] == True:
        x = Dense(hparams[HP_NUM_UNITS], activation="relu", name="h5")(x)
    
    cont_decoder_outputs = Dense(30, activation="linear", name="cont_decoder_output")(x)
    class_decoder_output = Dense(1, activation="sigmoid", name="classification_output")(x)
    decoder = keras.Model(
        latent_inputs, [cont_decoder_outputs, class_decoder_output], name="decoder"
    )

    class VAE(keras.Model):
        def __init__(self, encoder, decoder, **kwargs):
            super().__init__(**kwargs)
            # Trackers for the data-data-generation process
            self.encoder = encoder
            self.decoder = decoder
            self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
            self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
            self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
            self.reconstruction_loss_class_tracker = keras.metrics.Mean(name="reconstruction_loss_class")
            self.mean_abs_diff_cont_tracker = keras.metrics.Mean(name="mean_abs_diff_cont")
            self.mean_abs_diff_class_tracker = keras.metrics.Mean(name="mean_abs_diff_class")
            self.mean_binary_tracker = keras.metrics.Mean(name="mean_binary_value")
            self.kl_div_tracker = keras.metrics.Mean(name="kl_divergence")

            # Trackers for the sampling-data-generation process
            self.total_loss_tracker_sample = keras.metrics.Mean(name="total_loss_sample")
            self.reconstruction_loss_tracker_sample = keras.metrics.Mean(name="reconstruction_loss_sample")
            self.reconstruction_loss_class_tracker_sample = keras.metrics.Mean(name="reconstruction_loss_class_sample")
            self.mean_abs_diff_cont_tracker_sample = keras.metrics.Mean(name="mean_abs_diff_cont_sample")
            self.mean_abs_diff_class_tracker_sample = keras.metrics.Mean(name="mean_abs_diff_class_sample")
            self.mean_binary_tracker_sample = keras.metrics.Mean(name="mean_binary_value_sample")
            self.kl_div_tracker_sample = keras.metrics.Mean(name="kl_divergence_sample")

        @property
        def metrics(self):
            return [
                self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.kl_loss_tracker,
                self.reconstruction_loss_class_tracker,
            ]

        def train_step(self, data):
            with tf.GradientTape() as tape:
                z_mean, z_log_var, z = self.encoder(data)

                # latent_vectors = np.random.normal(scale=1, size=(32, latent_dim))
                # reconstruction_cont, reconstruction_class = self.decoder(latent_vectors)
                reconstruction_cont, reconstruction_class = self.decoder(z)

                data_cont = data[
                    :, :30
                ]  # Assuming the first 4 columns are for continuous variables
                data_class = data[:, 30:]  # Assuming the last column is for classification

                # Reconstruction loss for continuous outputs
                reconstruction_loss_cont = keras.losses.mean_squared_error(
                    data_cont, reconstruction_cont
                )

                # Reconstruction loss for classification output
                reconstruction_loss_class = keras.losses.binary_crossentropy(
                    data_class, reconstruction_class
                )

                kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
                kl_loss = tf.reduce_mean(kl_loss, axis=1)

                kl_div = tf.keras.losses.KLDivergence()(data_cont, reconstruction_cont)

                # Combine losses
                total_loss = (
                    reconstruction_loss_cont
                    + hparams[HP_BINARY_WEIGHT] * reconstruction_loss_class
                    + hparams[HP_KL_LOSS_RATE] * kl_loss
                )

            grads = tape.gradient(total_loss, self.trainable_weights)
            self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
            self.total_loss_tracker.update_state(total_loss)
            self.reconstruction_loss_tracker.update_state(reconstruction_loss_cont)
            self.kl_loss_tracker.update_state(kl_loss)
            self.reconstruction_loss_class_tracker.update_state(reconstruction_loss_class)
            self.kl_div_tracker.update_state(kl_div)

            return {
                "loss": self.total_loss_tracker.result(),
                "reconstruction_loss_cont": self.reconstruction_loss_tracker.result(),
                "reconstruction_loss_class": self.reconstruction_loss_class_tracker.result(),
                "kl_loss": self.kl_loss_tracker.result(),
                "kl_div": self.kl_div_tracker.result(),
            }

        def test_step(self, data):
            """Step run during validation."""
            if isinstance(data, tuple):
                data = data[0]  # Unpack the tuple and take the input data

            data_cont = data[:, :30]
            data_class = data[:, 30:]

            # The whole teststep for the data creation with data
            z_mean, z_log_var, z = self.encoder(data)

            reconstruction_cont, reconstruction_class = self.decoder(z)

            reconstruction_loss_cont = keras.losses.mean_squared_error(
                data_cont, reconstruction_cont
            )

            # Reconstruction loss for classification output
            reconstruction_loss_class = keras.losses.binary_crossentropy(
                data_class, reconstruction_class
            )

            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(kl_loss, axis=1)

            # Combine losses
            total_loss = (reconstruction_loss_cont
                    + hparams[HP_BINARY_WEIGHT] * reconstruction_loss_class
                    + hparams[HP_KL_LOSS_RATE] * kl_loss)

            # Compute mean absolute differences and mean of binary data
            mean_abs_diff_cont = tf.reduce_mean(tf.abs(data_cont - reconstruction_cont))
            mean_binary_value = tf.reduce_mean(
                tf.cast(reconstruction_class >= 0.5, tf.float32)
            )
            mean_abs_diff_class = tf.reduce_mean(tf.abs(data_class - mean_binary_value))

            # Compute the KL Div between original data and reconstructed
            kl_div = tf.keras.losses.KLDivergence()(data_cont, reconstruction_cont)

            # Whole test step for data creation with sampeling
            latent_vectors = np.random.normal(scale=1, size=(32, latent_dim))
            reconstruction_cont_sample, reconstruction_class_sample = self.decoder(
                latent_vectors
            )

            reconstruction_loss_cont_sample = keras.losses.mean_squared_error(
                data_cont, reconstruction_cont_sample
            )

            # Reconstruction loss for classification output
            reconstruction_loss_class_sample = keras.losses.binary_crossentropy(
                data_class, reconstruction_class_sample
            )

            # Combine losses
            total_loss_sample = (reconstruction_loss_cont_sample
                + hparams[HP_BINARY_WEIGHT] * reconstruction_loss_class_sample
                + hparams[HP_KL_LOSS_RATE] * kl_loss)

            # Compute mean absolute differences and mean of binary data
            mean_abs_diff_cont_sample = tf.reduce_mean(
                tf.abs(data_cont - reconstruction_cont_sample)
            )
            mean_binary_value_sample = tf.reduce_mean(
                tf.cast(reconstruction_class_sample >= 0.5, tf.float32)
            )
            mean_abs_diff_class_sample = tf.reduce_mean(
                tf.abs(data_class - mean_binary_value_sample)
            )

            # Compute the KL Div between original data and reconstructed
            kl_div_sample = tf.keras.losses.KLDivergence()(
                data_cont, reconstruction_cont_sample
            )

            # Update metrics
            # Update for data creation process
            self.total_loss_tracker.update_state(total_loss)
            self.reconstruction_loss_tracker.update_state(reconstruction_loss_cont)
            self.kl_loss_tracker.update_state(kl_loss)
            self.reconstruction_loss_class_tracker.update_state(reconstruction_loss_class)
            self.mean_abs_diff_cont_tracker.update_state(mean_abs_diff_cont)
            self.mean_abs_diff_class_tracker.update_state(mean_abs_diff_class)
            self.mean_binary_tracker.update_state(mean_binary_value)
            self.kl_div_tracker.update_state(kl_div)

            # Update for sampling creation process
            self.total_loss_tracker_sample.update_state(total_loss_sample)
            self.reconstruction_loss_tracker_sample.update_state(
                reconstruction_loss_cont_sample
            )
            self.reconstruction_loss_class_tracker_sample.update_state(
                reconstruction_loss_class_sample
            )
            self.mean_abs_diff_cont_tracker_sample.update_state(mean_abs_diff_cont_sample)
            self.mean_abs_diff_class_tracker_sample.update_state(mean_abs_diff_class_sample)
            self.mean_binary_tracker_sample.update_state(mean_binary_value_sample)
            self.kl_div_tracker_sample.update_state(kl_div_sample)

            # Return a dict mapping metric names to current value
            return {
                "loss": self.total_loss_tracker.result(),
                "reconstruction_loss_cont": self.reconstruction_loss_tracker.result(),
                "reconstruction_loss_class": self.reconstruction_loss_class_tracker.result(),
                "kl_loss": self.kl_loss_tracker.result(),
                "mean_abs_diff_cont": self.mean_abs_diff_cont_tracker.result(),
                "mean_abs_diff_class": self.mean_abs_diff_class_tracker.result(),
                "mean_binary_value": self.mean_binary_tracker.result(),
                "kl_div": self.kl_div_tracker.result(),

                # returning the sampling process testing
                "loss_sample": self.reconstruction_loss_tracker_sample.result(),
                "reconstruction_loss_cont_sample": self.reconstruction_loss_tracker_sample.result(),
                "reconstruction_loss_class_sample": self.reconstruction_loss_class_tracker_sample.result(),
                "mean_abs_diff_cont_sample": self.mean_abs_diff_cont_tracker_sample.result(),
                "mean_abs_diff_class_sample": self.mean_abs_diff_class_tracker_sample.result(),
                "mean_binary_value_sample": self.mean_binary_tracker_sample.result(),
                "kl_div_sample": self.kl_div_tracker_sample.result(),
            }

    log_dir = "logs/"
    vae = VAE(encoder, decoder)
    optimizer = optimizers.Adam()
    vae.compile(optimizer=optimizer)
    hist = vae.fit(
        train_data_array,
        epochs=1,
        batch_size=32,
        validation_data=(val_data_array, None),
        callbacks=[
            tf.keras.callbacks.TensorBoard(log_dir),  # log metrics
            hp.KerasCallback(log_dir, hparams),  # log hparams
        ],
    )
    return (
        hist.history["val_loss"],
        hist.history["val_mean_binary_value"],
        hist.history["val_mean_abs_diff_cont"],
        hist.history["val_mean_abs_diff_class"],
        hist.history["val_kl_loss"],
        hist.history["val_kl_div"],
        hist.history["val_loss_sample"],
        hist.history["val_mean_binary_value_sample"],
        hist.history["val_mean_abs_diff_cont_sample"],
        hist.history["val_mean_abs_diff_class_sample"],
        hist.history["val_kl_div_sample"],
    )

In [8]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        (
            val_loss,
            val_mean,
            val_mean_diff_cont,
            val_mean_diff_class,
            val_kl_loss,
            val_kl_div,
            val_loss_sample,
            val_mean_sample,
            val_mean_abs_diff_cont_sample,
            val_mean_abs_diff_class_sample,
            val_kl_div_sample,
        ) = build_model(
            hparams
        )  # Returns both val_loss and val_mean

        tf.summary.scalar(VAL_LOSS, val_loss[0], step=1)
        tf.summary.scalar(VAL_MEAN_BINARY_VALUE, val_mean[0], step=1)
        tf.summary.scalar(VAL_MEAN_CONT_DIFF , val_mean_diff_cont[0], step=1)
        tf.summary.scalar(VAL_MEAN_CLASS_DIFF, val_mean_diff_class[0], step=1)
        tf.summary.scalar(VAL_MEAN_KL_LOSS, val_kl_loss[0], step=1)
        tf.summary.scalar(VAL_MEAN_KL_DIV, val_kl_div[0], step=1)
        tf.summary.scalar(VAL_LOSS_SAMPLE, val_loss_sample[0], step=1)
        tf.summary.scalar(VAL_MEAN_BINARY_VALUE_SAMPLE, val_mean_sample[0], step=1)
        tf.summary.scalar(VAL_MEAN_CONT_DIFF_SAMPLE, val_mean_abs_diff_cont_sample[0], step=1)
        tf.summary.scalar(VAL_MEAN_CLASS_DIFF_SAMPLE, val_mean_abs_diff_class_sample[0], step=1)
        tf.summary.scalar(VAL_MEAN_KL_DIV_SAMPLE, val_kl_div_sample[0], step=1)

In [9]:
import os

# Define the logs directory
logs_dir = path

# Check if the logs directory exists
if os.path.exists(logs_dir):
    # Get a list of all files in the logs directory
    log_files = os.listdir(logs_dir)

    # Filter the list to only include files with the naming convention "run-<number>"
    run_numbers = [
        int(file.split("-")[1]) for file in log_files if file.startswith("run-")
    ]

    # Determine the next run number by finding the maximum run number and incrementing it by 1
    next_run_num = max(run_numbers) + 1 if run_numbers else 0
else:
    # If the logs directory doesn't exist, start from run-0
    next_run_num = 0

print("Next run number:", next_run_num)

Next run number: 0


In [10]:
session_num = next_run_num

for embedded in HP_NUM_embedded.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for num_units1 in HP_NUM_UNITS1.domain.values:
            for bool_2 in HP_BOOL_2_LAYER.domain.values:
                for bool_4 in HP_BOOL_4_LAYER.domain.values:
                    for beta in HP_KL_LOSS_RATE.domain.values:
                        for binary_value in HP_BINARY_WEIGHT.domain.values:
                            
                            hparams = {
                            HP_NUM_embedded: embedded,
                            HP_NUM_UNITS: num_units,
                            HP_NUM_UNITS1: num_units1,
                            #HP_BOOL_BATCH: bool_batch,
                            HP_BOOL_2_LAYER: bool_2,
                            HP_BOOL_4_LAYER: bool_4,
                            HP_KL_LOSS_RATE: beta,
                            HP_BINARY_WEIGHT: binary_value,
                        }
                            run_name = "run-%d" % session_num
                            print("--- Starting trial: %s" % run_name)
                            print({h.name: hparams[h] for h in hparams})
                            run(
                                path + run_name,
                                hparams,
                            )
                            session_num += 1


--- Starting trial: run-0
{'num_embedded': 2, 'num_units': 64, 'num_units1': 64, 'bool_layer_2': False, 'bool_layer_4': False, 'KL_LOSS_RATE': 0.0001, 'BINARY_WEIGHT': 0.1}
890/890 [==============================] - 4s 4ms/step - loss: 0.0423 - reconstruction_loss_cont: 0.0307 - reconstruction_loss_class: 0.0378 - kl_loss: 5.7548 - kl_div: 8.0970 - val_loss: 0.0111 - val_reconstruction_loss_cont: 0.0101 - val_reconstruction_loss_class: 0.0042 - val_kl_loss: 5.3583 - val_mean_abs_diff_cont: 0.0421 - val_mean_abs_diff_class: 0.0015 - val_mean_binary_value: 0.0000e+00 - val_kl_div: 1.4325 - val_loss_sample: 0.1613 - val_reconstruction_loss_cont_sample: 0.1613 - val_reconstruction_loss_class_sample: 0.3097 - val_mean_abs_diff_cont_sample: 0.3466 - val_mean_abs_diff_class_sample: 0.0015 - val_mean_binary_value_sample: 0.0000e+00 - val_kl_div_sample: 35.9154
--- Starting trial: run-1
{'num_embedded': 2, 'num_units': 64, 'num_units1': 64, 'bool_layer_2': False, 'bool_layer_4': False, 'KL_LOSS